In [36]:
import datetime
import imutils
import cv2       

In [37]:
cap = cv2.VideoCapture('cartheft2.mp4')
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('handmovement.mp4')

In [38]:
firstframe = None

In [39]:
while True:
    (grabbed, frame) = cap.read()
    text = ""
    
    if not grabbed:
        break
        
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    if firstframe is None:
        firstframe = gray
        continue
        
    frameDelta = cv2.absdiff(firstframe, gray)
    ret,thresh = cv2.threshold(frameDelta,50,255,cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)
    #image, contours, hierarchy  =   cv2.findContours(thresh.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    image, contours, hierarchy  =   cv2.findContours(thresh.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 500:
            continue
 
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.rectangle(frameDelta, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "MOVEMENT DETECTED"
        
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Status: {}".format(text), (10, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
                (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
 
    # show the frame and record if the user presses a key
    cv2.imshow("Original Video", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Difference", frameDelta)
    key = cv2.waitKey(1) & 0xFF
 
    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break

In [40]:
cap.release()
cv2.destroyAllWindows()